# Practical 03 - Bug1

In [ ]:
from Practical03_Support.Obstacle import *
from Practical03_Support.path_animation import *
import meshcat.geometry as g
import meshcat.transformations as tf

from ece4078.Utility import StartMeshcat
from shapely import geometry

In [ ]:
vis = StartMeshcat()

## Helper functions
These are the helper functions used by Bug1 when looking for a path between the start and goal locations.

In [ ]:
# Helper functions
def find_closest_obstacle(position, obstacle_list, ccw):
    results = [obs.compute_distance_point_to_polygon(position, ccw) for obs in obstacle_list]
    closest_obs = np.argmin([v[0] for v in results])
    return obstacle_list[closest_obs], results[closest_obs]

def has_reached_goal(current_pos, goal, step_size):
    if compute_distance_between_points(current_pos, goal) > step_size:
        return False
    return True

def move_towards_goal(current_pos, goal_pos, step_size):
    direction_to_goal = get_direction_from_points(current_pos, goal_pos)
    next_position = current_pos + step_size * direction_to_goal
    return next_position

def is_about_to_hit_obstacle(next_pos, obstacle_list, step_size, ccw):
    obs, (_, _) = find_closest_obstacle(next_pos, obstacle_list, ccw)
    point = geometry.Point(next_pos)
    polygon = geometry.Polygon(obs.vertices)
    if point.within(polygon) or point.touches(point):
        return True
    else:
        return False

Taking the code of Bug2 algorithm as a starting point, you are asked to implement the Bug1 algorithm (shown below)

![title](https://i.imgur.com/VdNT8c0.png)

A few pointers, within the main loop, we will have two loops:
- A first loop to find the point $Q$ along the obstacle's boundary that is the closest to the goal.
- A second loop that circumnavigates the obstacle until $Q$ is reached again
- Once the robot has reached $Q$, it should come back to the main loop and move towards the goal

Ideas: 
- Use the ``hit_point`` to know when a first path along the obstacle's boundary has been completed
- Keep in memory (using the lists ``boundary_points, goal_distances = [], []``) of all positions and distances to goal along the obstacle's boundary. Use this memory to determine whether the robot has reached $Q$

In [ ]:
# Implement Bug1 Algorithm
def bug1_algorithm(goal_pos,initial_robot_pos,robot_step_size,obstacles,ccw):
    # Start algorithm
    robot_path = [initial_robot_pos]
    current_robot_pos = initial_robot_pos 
    robot_to_goal_line = compute_line_through_points(current_robot_pos, goal_pos)

    # While goal not reached
    while not has_reached_goal(current_robot_pos, goal_pos, robot_step_size):

        # Move towards goal
        next_robot_pos = move_towards_goal(current_robot_pos, goal_pos, robot_step_size)
        hit_point = np.copy(next_robot_pos)

        if is_about_to_hit_obstacle(next_robot_pos, obstacles, robot_step_size, ccw):
            # Go back to current position
            next_robot_pos = current_robot_pos
            # Compute distance from hit point to goal        
            boundary_points, goal_distances = [], []

            while True:
                closest_obs, (closest_obs_distance, obst_segment) = find_closest_obstacle(next_robot_pos,
                                                                                         obstacles, ccw)
                # Get direction along obstacle
                direction_around_obstacle = closest_obs.compute_tangent_vector_to_polygon(next_robot_pos, 
                                                                                          obst_segment)
                # Move along obstacle
                next_robot_pos = next_robot_pos + robot_step_size * direction_around_obstacle
                robot_path.append(next_robot_pos)

                goal_distances.append(compute_distance_between_points(next_robot_pos, goal_pos))
                boundary_points.append(next_robot_pos)

                completed_tour = (compute_distance_between_points(hit_point, next_robot_pos) < robot_step_size)

                if completed_tour:
                    while True:
                        idx_min_dist = np.argmin(goal_distances)

                        # Find nearest_obstacle and start navigating around 
                        closest_obs, (closest_obs_distance, obst_segment) = find_closest_obstacle(next_robot_pos,
                                                                                                 obstacles,ccw)
                        # Get direction along obstacle
                        direction_around_obstacle = closest_obs.compute_tangent_vector_to_polygon(next_robot_pos, 
                                                                                                  obst_segment)
                        # Move along obstacle
                        next_robot_pos = next_robot_pos + robot_step_size * direction_around_obstacle

                        robot_path.append(next_robot_pos)

                        # Stop getting around obstacle when we found point closest to goal:
                        stop_getting_around = compute_distance_between_points(next_robot_pos,
                                                                              goal_pos) <= goal_distances[idx_min_dist]

                        if stop_getting_around:
                            break
                    break

        # Update current state and add to path
        current_robot_pos = next_robot_pos
        robot_path.append(current_robot_pos)

    path = np.array(robot_path)
    return path

In [ ]:
# Set parameters

# # Order A
goal_pos = [20.0, 10.0]
initial_robot_pos = [30.0, 40.0]

# # Order B
# # Uncomment the next three lines to see the bug algorithm in reverse order
# goal_pos = [30.0, 40.0]
# initial_robot_pos = [20.0, 10.0]

# Robot Step Size and turning behaviour (counter clockwise (ccw) or clockwise)
robot_step_size = 0.2
ccw = True

# Obstacle configuration
obstacle1 = Rectangle(origin=np.array([10, 20]), width=30, height=5)
obstacle2 = Polygon(vertices= np.array([[20, 35], [35, 35], [35, 50], [40, 50],
                                       [40, 30], [20, 30]]))
obstacles = [obstacle1, obstacle2]

path = bug1_algorithm(goal_pos,initial_robot_pos,robot_step_size,obstacles,ccw)
steps = len(path)

##  Test your Code

Use the code provided below to visually test your implementation. If your solution is correct you should obtain a path similar(but not exact) to this one. 

<img src="https://i.imgur.com/0L0IhZE.gif" height="400" width="600" align="center">

In [ ]:
thickness = 0.5
robot_size = 0.5
vis.delete()
scale = 60
vis.Set2DView(scale, center = [lim for lim in [-30, 60, 60, 0]])

print("animating ...")
animate_path_bug(vis,initial_robot_pos,goal_pos,path,obstacles, robot_size, thickness, goal_line = False)

import ipywidgets as widgets
msg = widgets.HTML(f"<big>Total of steps: {steps}</big>")
display(widgets.VBox([msg, vis.show_inline(height = 500)]))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8ca87fd4-86a8-4e2a-9521-da2ddccb6e1a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>